## Key Probabilistic Layers in `probly`

### 1. `BayesLinear` & `BayesConv2d`

**What they do:**  
These layers replace standard `Linear` and `Conv2d` layers to build a **Bayesian Neural Network (BNN)**.

**How they work:**  
Instead of using fixed weights, each weight is modeled as a probability distribution (e.g., Gaussian).  
On every forward pass, weights are sampled from these distributions.

**Result:**  
The model can explicitly represent **uncertainty in its own parameters**.

In [2]:
import torch
from torch import nn

from probly.layers.torch import BayesLinear, DropConnectLinear, NormalInverseGammaLinear

# 1. Define a standard nn.Linear layer to serve as a base
base_linear = nn.Linear(in_features=10, out_features=5)

# 2. Create a BayesLinear layer from the base layer
bayes_linear = BayesLinear(base_linear)

# 3. Create a dummy input tensor
dummy_input = torch.randn(1, 10)

# 4. Demonstrate stochastic forward passes
# Note: No need to call .train() as Bayesian layers are always stochastic
print("--- Stochastic Forward Passes ---")
output1 = bayes_linear(dummy_input)
output2 = bayes_linear(dummy_input)

print("Output of first pass:\n", output1)
print("\nOutput of second pass:\n", output2)
print(f"\nOutputs are the same: {torch.allclose(output1, output2)}")

# 5. Access the KL divergence property (crucial for training BNNs)
kl = bayes_linear.kl_divergence
print(f"\nCalculated KL Divergence: {kl.item():.4f}")

--- Stochastic Forward Passes ---
Output of first pass:
 tensor([[-0.0915, -0.0420, -0.8667,  0.2994,  0.3818]],
       grad_fn=<AddmmBackward0>)

Output of second pass:
 tensor([[-0.2573,  0.1226, -0.4881,  0.4306,  0.5179]],
       grad_fn=<AddmmBackward0>)

Outputs are the same: False

Calculated KL Divergence: 138.1445



### 2. `DropConnectLinear`

**What it does:**  
Implements the **DropConnect** technique as a neural network layer.

**How it works:**  
During training (and during inference when performing uncertainty quantification), individual weights are randomly set to zero.  
This is a more general form of Dropout.

**Result:**  
Running multiple forward passes produces a **distribution of outputs** that captures model uncertainty.

In [3]:
# 1. Define a standard nn.Linear layer as a base
base_linear = nn.Linear(in_features=20, out_features=10)

# 2. Create a DropConnectLinear layer with a 50% drop probability
dc_linear = DropConnectLinear(base_linear, p=0.5)

# 3. Create a dummy input tensor
dummy_input = torch.randn(1, 20)

# 4. Demonstrate stochastic behavior in .train() mode
print("--- Behavior in .train() mode ---")
dc_linear.train()
output_train1 = dc_linear(dummy_input)
output_train2 = dc_linear(dummy_input)
print("Output of first pass:\n", output_train1)
print("\nOutput of second pass:\n", output_train2)
print(f"\nOutputs are the same: {torch.allclose(output_train1, output_train2)}")

# 5. Demonstrate deterministic behavior in .eval() mode
print("\n\n--- Behavior in .eval() mode ---")
dc_linear.eval()
output_eval1 = dc_linear(dummy_input)
output_eval2 = dc_linear(dummy_input)
print("Output of first pass:\n", output_eval1)
print("\nOutput of second pass:\n", output_eval2)
print(f"\nOutputs are the same: {torch.allclose(output_eval1, output_eval2)}")

--- Behavior in .train() mode ---
Output of first pass:
 tensor([[-0.3468,  0.1877, -0.2025,  0.2372, -0.2160,  0.0782, -0.0573, -0.0387,
         -0.4009,  0.4548]], grad_fn=<AddmmBackward0>)

Output of second pass:
 tensor([[-0.1440, -0.0132, -0.3005,  0.4418,  0.2543, -0.0857, -0.3065, -0.4581,
         -0.9260,  1.1667]], grad_fn=<AddmmBackward0>)

Outputs are the same: False


--- Behavior in .eval() mode ---
Output of first pass:
 tensor([[-0.3639,  0.0404, -0.1381,  0.2764, -0.0017,  0.0430, -0.0628,  0.1629,
         -0.5623,  0.5159]], grad_fn=<AddmmBackward0>)

Output of second pass:
 tensor([[-0.3639,  0.0404, -0.1381,  0.2764, -0.0017,  0.0430, -0.0628,  0.1629,
         -0.5623,  0.5159]], grad_fn=<AddmmBackward0>)

Outputs are the same: True



### 3. `NormalInverseGammaLinear` (for Evidential Regression)

**What it does:**  
A specialized output layer designed for **Evidential Regression**.

**How it works:**  
Instead of predicting a single value, the layer outputs the four parameters of a **Normal-Inverse-Gamma (NIG)** distribution:

- `gamma`
- `nu`
- `alpha`
- `beta`

**Result:**  
This distribution directly models:

- The predictive mean  
- The predictive variance  
- The model’s confidence in its own predictions


In [4]:
# 1. Create the NIG layer directly (it does not take a base layer)
# For a regression task, out_features is typically 1
nig_linear = NormalInverseGammaLinear(in_features=10, out_features=1)

# 2. Create a dummy input tensor
dummy_input = torch.randn(1, 10)

# 3. Perform a single forward pass
params = nig_linear(dummy_input)

# 4. Inspect the output dictionary
print("--- Output Dictionary ---")
print(params)

print("\n\n--- Individual Parameters ---")
print("Gamma (prediction):", params["gamma"])
print("Nu (variance regularizer):", params["nu"])
print("Alpha (precision):", params["alpha"])
print("Beta (variance):", params["beta"])

--- Output Dictionary ---
{'gamma': tensor([[-1.7726]], grad_fn=<AddmmBackward0>), 'nu': tensor([[0.5928]], grad_fn=<SoftplusBackward0>), 'alpha': tensor([[1.5074]], grad_fn=<AddBackward0>), 'beta': tensor([[0.5090]], grad_fn=<SoftplusBackward0>)}


--- Individual Parameters ---
Gamma (prediction): tensor([[-1.7726]], grad_fn=<AddmmBackward0>)
Nu (variance regularizer): tensor([[0.5928]], grad_fn=<SoftplusBackward0>)
Alpha (precision): tensor([[1.5074]], grad_fn=<AddBackward0>)
Beta (variance): tensor([[0.5090]], grad_fn=<SoftplusBackward0>)
